# Projeto Fase 6 – Rede Neural (YOLOv5 e Comparativos)

Este notebook é a implementação da Fase 6 do projeto de IA da FIAP para o grupo **rede_neural_fase6**. O objetivo é demonstrar o desenvolvimento de um sistema de visão computacional utilizando **YOLOv5**, avaliando sua performance e comparando com abordagens alternativas:

- YOLOv5 customizado para reconhecer dois objetos (A e B) a partir de um dataset próprio.
- Detecção com YOLOv5 pré-treinado (baseline zero-shot).
- Classificação de imagens com uma **CNN treinada do zero**.

Prepare seu ambiente seguindo as instruções e execute cada etapa em sequência. Onde indicado, substitua pelos caminhos corretos para suas imagens no Google Drive.

In [ ]:

# ## 1. Conectar ao Google Drive e preparar o dataset

from google.colab import drive
import os, glob, shutil, random
from pathlib import Path

# Montar o Google Drive (será solicitado a autorização)
drive.mount('/content/drive')

# ====== CONFIGURAÇÕES ======
# Ajuste estes caminhos para as pastas no seu Drive contendo as imagens de cada classe
ORIGEM_IMAGENS_A = '/content/drive/MyDrive/SEU_CAMINHO/objeto_A_imagens'  # substitua
ORIGEM_IMAGENS_B = '/content/drive/MyDrive/SEU_CAMINHO/objeto_B_imagens'  # substitua
# Se já anotou com MakeSense e tem arquivos .txt YOLO, ajuste os caminhos das labels
ORIGEM_LABELS_A  = '/content/drive/MyDrive/SEU_CAMINHO/objeto_A_labels'   # ou deixe como '' se ainda não houver
ORIGEM_LABELS_B  = '/content/drive/MyDrive/SEU_CAMINHO/objeto_B_labels'   # ou deixe como '' se ainda não houver

# Nomes das classes
CLASSES = ['objeto_A', 'objeto_B']

# Quantidade mínima de imagens por classe (80 no total)
N_TRAIN = 32
N_VAL   = 4
N_TEST  = 4

# Criar estrutura de pastas do dataset
for p in ['dataset/images/train','dataset/images/val','dataset/images/test',
          'dataset/labels/train','dataset/labels/val','dataset/labels/test']:
    os.makedirs(p, exist_ok=True)

# Função para coletar imagens
def collect_images(path, patterns=('*.jpg','*.jpeg','*.png')):
    files = []
    for pat in patterns:
        files += glob.glob(os.path.join(path, '**', pat), recursive=True)
    return sorted(files)

# Coletar imagens de cada classe
imgs_A = collect_images(ORIGEM_IMAGENS_A)
imgs_B = collect_images(ORIGEM_IMAGENS_B)

print(f'Classe A: {len(imgs_A)} imagens, Classe B: {len(imgs_B)} imagens')
assert len(imgs_A) >= (N_TRAIN+N_VAL+N_TEST) and len(imgs_B) >= (N_TRAIN+N_VAL+N_TEST), 'Quantidade insuficiente de imagens para o mínimo exigido.'

# Embaralhar e separar imagens em treino, validação e teste
random.seed(42)
random.shuffle(imgs_A)
random.shuffle(imgs_B)

splits = {'train': N_TRAIN, 'val': N_VAL, 'test': N_TEST}

def distribute(images, labels_dir):
    offset = 0
    for split, qty in splits.items():
        part = images[offset:offset+qty]
        offset += qty
        for img in part:
            dest_img = f"dataset/images/{split}/" + os.path.basename(img)
            shutil.copy(img, dest_img)
            # copiar label correspondente, se existir
            if labels_dir:
                label_filename = os.path.splitext(os.path.basename(img))[0] + '.txt'
                label_path = os.path.join(labels_dir, label_filename)
                if os.path.exists(label_path):
                    shutil.copy(label_path, f"dataset/labels/{split}/" + label_filename)

# Distribuir imagens e (opcionalmente) labels
distribute(imgs_A, ORIGEM_LABELS_A)
distribute(imgs_B, ORIGEM_LABELS_B)

print('Imagens copiadas para dataset/images e labels (se houver).')

# Gerar arquivo dataset.yaml para YOLOv5
yaml_content = f'# Dataset personalizado
path: .
train: dataset/images/train
val: dataset/images/val
test: dataset/images/test
names:
  0: {CLASSES[0]}
  1: {CLASSES[1]}
'
with open('dataset.yaml', 'w') as f:
    f.write(yaml_content)
print(open('dataset.yaml').read())


In [ ]:

# ## 2. Instalar YOLOv5

# Clonar o repositório do YOLOv5 e instalar dependências
!git clone -q https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt
print('YOLOv5 instalado.')


In [ ]:

# ## 3. Treinamento do modelo (30 e 60 épocas)

import time, glob, os

runs_info = []

def train_yolo(epochs, run_name):
    start = time.time()
    !python train.py --img 640 --batch 16 --epochs {epochs} --data /content/dataset.yaml --weights yolov5s.pt --name {run_name} --project runs/detect
    duration = time.time() - start
    exp_dir = sorted(glob.glob(f'runs/detect/{run_name}*'))[-1]
    runs_info.append({'run': os.path.basename(exp_dir), 'epochs': epochs, 'duration_s': duration})

# Treinar com 30 e 60 épocas
train_yolo(30, 'fase6_e30')
train_yolo(60, 'fase6_e60')

print('Treinamentos concluídos. Resumo:', runs_info)


In [ ]:

# ## 4. Comparar métricas dos modelos

import pandas as pd, glob, os
from pathlib import Path

def extract_metrics(exp_dir):
    csv_files = glob.glob(os.path.join(exp_dir, 'results.csv'))
    if csv_files:
        df = pd.read_csv(csv_files[0])
        last_row = df.iloc[-1]
        return {
            'run': Path(exp_dir).name,
            'epochs': int(last_row.get('epoch', 0)),
            'mAP@0.5': float(last_row.get('mAP_0.5', last_row.get('metrics/mAP_0.5', 0))),
            'mAP@0.5:0.95': float(last_row.get('mAP_0.5:0.95', last_row.get('metrics/mAP_0.5:0.95', 0))),
            'precision': float(last_row.get('precision', last_row.get('metrics/precision', 0))),
            'recall': float(last_row.get('recall', last_row.get('metrics/recall', 0))),
        }
    return None

metrics_list = []
for exp in glob.glob('runs/detect/fase6_e*'):
    m = extract_metrics(exp)
    if m:
        metrics_list.append(m)

pd.DataFrame(metrics_list).sort_values('mAP@0.5:0.95', ascending=False)


In [ ]:

# ## 5. Inferência no conjunto de teste e salvar imagens

import glob, os

best_run_dir = 'runs/detect/fase6_e60'  # ajuste conforme avaliação
best_weights = glob.glob(os.path.join(best_run_dir, 'weights', 'best.pt'))[0]

!python detect.py --weights {best_weights} --img 640 --source '/content/dataset/images/test' --name 'fase6_test_pred' --project runs/predict --save-txt --save-conf
print('Imagens de teste processadas. Confira a pasta runs/predict/fase6_test_pred* para visualizar.')


In [ ]:

# ## 6. Baseline YOLOv5 pré-treinado (zero-shot)

!python detect.py --weights yolov5s.pt --img 640 --source '/content/dataset/images/test' --name 'baseline_zeroshot' --project runs/predict
print('Baseline concluído. Veja runs/predict/baseline_zeroshot* para resultados.')


In [ ]:

# ## 7. Classificação de imagens com CNN do zero

import tensorflow as tf
from tensorflow.keras import layers, models
import pathlib, os, shutil
from glob import glob
from pathlib import Path

# Definir classes
CLASSES = ['objeto_A', 'objeto_B']

# Organizar conjunto de dados em pastas por classe
base_cls = Path('dataset_cls')
for split in ['train','val','test']:
    for cls in CLASSES:
        os.makedirs(base_cls / split / cls, exist_ok=True)

# Função para mover imagens

def move_to_classification(split, class_name, images):
    for img in images:
        dest = base_cls / split / class_name / Path(img).name
        if not dest.exists():
            shutil.copy(img, dest)

# Selecionar imagens de cada classe
a_train = glob('dataset/images/train/*')
v_train = glob('dataset/images/val/*')
t_test  = glob('dataset/images/test/*')

# Dividir metade/ metade (assume ordem embaralhada)
mid_train = len(a_train)//2
mid_val   = len(v_train)//2
mid_test  = len(t_test)//2

move_to_classification('train', CLASSES[0], a_train[:mid_train])
move_to_classification('train', CLASSES[1], a_train[mid_train:])
move_to_classification('val',   CLASSES[0], v_train[:mid_val])
move_to_classification('val',   CLASSES[1], v_train[mid_val:])
move_to_classification('test',  CLASSES[0], t_test[:mid_test])
move_to_classification('test',  CLASSES[1], t_test[mid_test:])

# Criar datasets do TensorFlow
img_size = (224, 224)
batch_size = 16
train_ds = tf.keras.preprocessing.image_dataset_from_directory(base_cls/'train', image_size=img_size, batch_size=batch_size, label_mode='categorical', seed=42)
val_ds   = tf.keras.preprocessing.image_dataset_from_directory(base_cls/'val',   image_size=img_size, batch_size=batch_size, label_mode='categorical', seed=42)
test_ds  = tf.keras.preprocessing.image_dataset_from_directory(base_cls/'test',  image_size=img_size, batch_size=batch_size, label_mode='categorical', seed=42)

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)

# Modelo simples
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=img_size + (3,)),
    layers.Conv2D(32, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(CLASSES), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, epochs=20, validation_data=val_ds, verbose=1)

loss, acc = model.evaluate(test_ds, verbose=0)
print(f'Acurácia no conjunto de teste: {acc:.4f}')


# 8. Conclusões e comparações

- **YOLOv5 customizado**: Utilizando 30 e 60 épocas, avaliamos o desempenho em termos de mAP e balanceamos precisão/recall.
  Mais épocas tendem a melhorar a performance, mas aumentam o tempo de treinamento. Analise os resultados para escolher o
  modelo mais equilibrado.
- **YOLOv5 pré-treinado (baseline)**: Aplicado diretamente sem ajustes, serve como controle para mostrar que ele não reconhece objetos específicos
  ausentes no COCO. Espera-se mAP baixo e detecções incorretas.
- **CNN do zero**: Treinada para classificar imagens A vs B. Útil para comparar detecção vs classificação. Fornece acurácia alta porém não
  localiza o objeto.

**Discussão**:

- *Facilidade de uso/integração*: YOLOv5 é relativamente simples de treinar e utilizar para detecção. A CNN de classificação é mais simples,
  mas limitada a saber o que sem indicar onde.
- *Precisão*: Compare a mAP das abordagens YOLO com a acurácia da CNN. O ajuste fino do YOLO customizado deve superar o baseline.
- *Tempo de treinamento*: YOLOv5 com 60 épocas leva aproximadamente o dobro do tempo de 30 épocas. A CNN é mais rápida, porém depende do tamanho do dataset.
- *Tempo de inferência*: YOLOv5 processa a localização e pode ser mais lento, enquanto a CNN classifica rapidamente.

Inclua prints de predições do YOLO customizado em imagens de teste na documentação e no vídeo final para convencer o cliente da eficácia do sistema.